In [ ]:
import pygame
from Functions import *
from Classes import *
from Settings import *

def main():
    pygame.init()
    screen = pygame.display.set_mode((ROOM_WIDTH, ROOM_HEIGHT))
    pygame.display.set_caption("MINESWEEPER PROJECT")
    clock = pygame.time.Clock()

    # initialize counters for my dictionary:
    wins = 0
    losses = 0

    while True:  # each iteration starts a new round, it will always be true so when restart is hit it breaks out the loop just to be ran again restarting the game, will only stop runnning if pygame is quit
        board = Board()
        board.display_board()
        running = True
        restart = False
        win_flag = False

        while running:
            clock.tick(FPS)

# Code will always check for quitting events which right before it does quit, it will print the dictionary
            for event in pygame.event.get():
                if event.type == pygame.QUIT: 
                    print({'Wins': wins, 'Losses': losses})
                    pygame.quit()
                    return

# Same as code above except for the escape key
                if event.type == pygame.KEYDOWN:
                    if event.key == pygame.K_ESCAPE:
                        print({'Wins': wins, 'Losses': losses})
                        pygame.quit()
                        return
                    if event.key == pygame.K_r:
                        restart = True
                        running = False
                        break

# Checks for if event is a mouse click, then takes coordinates of that click and basically does mx = mx // tile_size and my = my // tile_size
                if event.type == pygame.MOUSEBUTTONDOWN:
                    mx, my = pygame.mouse.get_pos()
                    mx //= TILE_SIZE
                    my //= TILE_SIZE # have to do floor division because in classes you took the x and multiplied it by the tilesize so now you have to compensate for that

                    if event.button == 1:
                        # if left click
                        if not board.board_list[mx][my].flagged: #checks if the tile in the list of lists is flagged
                            if not board.dig(mx, my): # if it's not flagged and not digged 
                                # exploded: reveal bombs and mark incorrect flags
                                for row in board.board_list:
                                    for tile in row:
                                        if tile.flagged and tile.type != "B":
                                            tile.flagged = False
                                            tile.revealed = True # logic for if the tile, the user clicked is flagged and not a bomb then unflag and reveal it
                                            tile.image = tile_not_mine
                                        elif tile.type == "B": # logic for if the user clicked a bomb then reveal the bomb and stop the game
                                            tile.revealed = True
                                running = False
                                win_flag = False
                    elif event.button == 3:
                        # right click -> toggle flag logic
                        if not board.board_list[mx][my].revealed:
                            board.board_list[mx][my].flagged = not board.board_list[mx][my].flagged #logic for if user right clicks a flagged tile, to unflag it
                

            # Only check win if still running
            if running:
                win = True
                for row in board.board_list: # logic to check if win = true and then if so set it to false
                    for tile in row:
                        if tile.type != "B" and not tile.revealed:
                            win = False
                            break
                    if not win:
                        break

                if win:
                    # mark all unrevealed as flagged to show win state
                    for row in board.board_list:
                        for tile in row:
                            if not tile.revealed:
                                tile.flagged = True #basically everything will show up as flags if you win and set win_flag to true and stop running
                    running = False
                    win_flag = True

            # Draw the board every frame
            screen.fill(BG_COLOR)
            board.draw(screen)
            pygame.display.flip()

        # Inner loop ended. If R was pressed during play, restart immediately.
        if restart:
            continue

        # Update wins/losses (update counters here so printed dict is current)
        if win_flag:
            wins += 1
        else:
            losses += 1

        # End screen: wait until player presses R (or Esc/quit)
        end_wait = True
        while end_wait:
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    print({'Wins': wins, 'Losses': losses})
                    pygame.quit()
                    return
                if event.type == pygame.KEYDOWN:
                    if event.key == pygame.K_ESCAPE:
                        # print current totals before quitting
                        print({'Wins': wins, 'Losses': losses})
                        show_game_over()
                        pygame.quit()
                        return
                    if event.key == pygame.K_r:
                        end_wait = False
                        break

        # loop continues and a new board will be created automatically

if __name__ == "__main__":
    main()